In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
path_1 = "/content/drive/My Drive/BERT_Anadi/Data Produced/bert_train_embed_1000_docs.pickle"
path_2 = "/content/drive/My Drive/BERT_Anadi/Data Produced/bert_gs_embed_all.pickle"


data = pd.read_pickle((path_1))
# data = data.drop(columns = ['doc_id'])
X_pred = (pd.read_pickle(path_2))

# gc.collect()

In [0]:
data.shape

(130759, 1026)

In [0]:
X_pred.shape

(9092, 1025)

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(91531, 1024) (91531,) (39228, 1024) (39228,)


In [0]:
print((y_train.value_counts()+y_test.value_counts())/(474108+168182))

In [0]:
X_pred.drop(columns = ['doc_id'], inplace = True)
X_pred.shape

(9092, 1024)

In [0]:
from sklearn.model_selection import RandomizedSearchCV
import time
from scipy.stats import uniform

model = LogisticRegression(max_iter=10000000)

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter distribution using uniform distribution
C = uniform(loc=0, scale=4)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

# Create randomized search 5-fold cross validation and 100 iterations
clf = RandomizedSearchCV(model, hyperparameters, random_state=42, n_iter=30, cv=3, verbose=1, n_jobs=-1, scoring = 'f1')

s = time.time()
best_model = clf.fit(X_train, y_train)
e = time.time()

# View best hyperparameters
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print("Model Operation Completed in :",(e-s)/60,"min")

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 10.5min finished


Best Penalty: l2
Best C: 3.795542149013333
Model Operation Completed in : 11.194270352522532 min


In [0]:
model = LogisticRegression(max_iter=10000000, penalty = best_model.best_estimator_.get_params()['penalty'], C = best_model.best_estimator_.get_params()['C'])

import time

s = time.time()
model.fit(X_train, y_train)
e = time.time()

print("Model trained in", (e-s)/60,"min")
print(model.score(X_test, y_test))

pred = model.predict(X_test)
print(f1_score(pred, y_test))

Model trained in 0.6670020778973897 min
0.7984857754665036
0.23997692529564466


In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []
acc = []

pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = cutoff, y = f1, mode='lines', name='f1'))
fig.add_trace(go.Scatter(x = cutoff, y = acc, mode='lines', name='accuracy'))
fig.show()